In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels= 1, out_channels= 6 , kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels= 6, out_channels= 12 , kernel_size=5)
        
        self.fc1 = nn.Linear(in_features= 12*4*4 , out_features= 120)
        self.fc2 = nn.Linear(in_features= 120, out_features= 60)
        self.out = nn.Linear(in_features= 60, out_features= 10)

    def forward(self,t):
        
        #1 inpput layer 
        
        t = t
        
        #2 hidden conv layer
        
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2 , stride = 2)
        
        #3 hidden conv layer 
        
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2 , stride = 2)
        
        #4 Linear layer 
        
        t = t.reshape(-1, 12*4*4)     #flattening is hapening here
        t = self.fc1(t)
        t = F.relu(t)

        
        #5 Linear layer 
        
        t = self.fc2(t)
        t = F.relu(t)
        
        #6 output layer
        
        t = self.out(t)
        # t = F.softmax(t,dim = 0)    but this line is not required because we will predict the output later and softmax will be used explicitly later
        return t
        
#loading data from url of Mnist

train_set = torchvision.datasets.FashionMNIST(
        root = './data'
        ,train = True
        ,download=True
        ,transform = transforms.Compose([
            transforms.ToTensor()
        ])
    
)

# Building two classes Runbuilder and Run manager 

In [5]:
class RunBuilder():
    @staticmethod
    
    def get_runs(params):
        
        
        Run = namedtuple('Run',params.keys())
        
        runs = []
        
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs
        

In [6]:
class RunManager():
    
    def __init__(self):
        
        self.epoch_count = 0               #initializing the class to keep track of some attributes mentioned here
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_count = 0                      #gives us the run number
        
        self.run_parameters = None         #This is the run definition in terms for the run parameters.
                                            #It's value will be one of the runs returned by the RunBuilder class."""
        
        self.run_data = []               #list we'll use to keep track of the parameter values and the results
                                            #of each epoch for each run, and so we'll see that we add a value to this list for each epoch"""
        
        self.run_start_time = None                 # calculate the run time
        
        self.network = None
        self.loader = None
        self.tb = None
    
        
        
    # now lets make the begin run and end run methodss here to describe whats going on
        
    def begin_run(self,run,network,loader):

        self.run_start_time = time.time()
        self.run_count +=1
        self.run_parameters = run

        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment= f'{run}')

        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images', grid)
        self.tb.add_graph(self.network, images)



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0    # the run is done for aparticular choice of hyperparameter so epochs will go to zero for the next run of other hyper parameters 



    #Now we define the methods for the begin epochs and end epochs


    def begin_epoch(self):

        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):

        epoch_delay = time.time() - self.epoch_start_time
        run_delay = time.time() - self.run_start_time
        
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy =self.epoch_num_correct/len(self.loader.dataset)
        self.tb.add_scalar("Loss", loss , self.epoch_count)
        self.tb.add_scalar("Acuracy", accuracy , self.epoch_count)
        
        for name,param in self.network.named_parameters():
            self.tb.add_histogram(name , param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad' , param.grad, self.epoch_count)
        
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = epoch_delay
        results['run duration'] = run_delay
        
        for k,v in self.run_parameters._asdict().items(): results[k] = v   #we iterate over the keys and values inside our run parameters adding them to the results dictionary. This will allow us to see the parameters that are associated with the performance results.
        self.run_data.append(results)
        
        df = pd.DataFrame.from_dict(self.run_data,orient = 'columns')
            
        clear_output(wait=True)                #for jupyter notebook only
        display(df)
        

    def track_loss(self,loss,batch):
        self.epoch_loss += loss.item()*batch[0].shape[0]
        
        

    def track_num_currect(self,preds,lables):
        self.epoch_num_correct += self._get_num_correct(preds,labels)
        

        
    def _get_num_correct(self,preds,labels):
        return preds.argmax(dim =1).eq(labels).sum().item()
    
    
    
    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data, orient='columns'
        ).to_csv(f'{fileName}.csv')

        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [ ]:
params = OrderedDict(
    lr = [.01,.001]
    ,batch_size = [1000, 10000],
    num_workers = [0,1,4]
    
)
epochs = 4
m = RunManager()
for run in RunBuilder.get_runs(params):
    my_net = Network()
    loader = torch.utils.data.DataLoader(train_set, batch_size= run.batch_size, num_workers = run.num_workers)
    optimizer = optim.Adam(my_net.parameters(), lr = run.lr)
    
    m.begin_run(run,my_net,loader)
    
    for epoch in range(epochs):
        m.begin_epoch()
        
        for batch in loader:
            images,labels = batch
            preds  = my_net(images)
            loss = F.cross_entropy(preds,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss,batch)
            m.track_num_currect(preds,labels)
        m.end_epoch()
    m.end_run()
m.save('first_run_results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers
0,1,1,1.020335,0.611400,12.995622,35.037864,0.01,1000,0
1,1,2,0.566733,0.780500,11.674317,46.916633,0.01,1000,0
2,1,3,0.466057,0.826583,12.923108,59.958214,0.01,1000,0
3,1,4,0.399585,0.854083,12.007528,72.071428,0.01,1000,0
4,2,1,0.875552,0.673050,8.622833,10.039491,0.01,1000,1
5,2,2,0.478960,0.822233,8.965144,19.148097,0.01,1000,1
6,2,3,0.393968,0.855867,9.748299,29.057053,0.01,1000,1
7,2,4,0.350546,0.871333,10.917571,40.172164,0.01,1000,1
8,3,1,0.958595,0.629567,12.420371,13.909063,0.01,1000,4
9,3,2,0.535456,0.786833,11.969656,26.044218,0.01,1000,4
